In [79]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense , Dropout, InputLayer
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from sklearn.model_selection import cross_val_score,KFold


import tensorflow as tf
import datetime
from tensorflow.keras.callbacks import TensorBoard

In [80]:
data=pd.read_csv('train_data_ready.csv')
x=data.iloc[:,7:22]
y=data.loc[:,'target']

In [87]:
model=Sequential()
model.add(InputLayer(input_shape=(15,)))
model.add(Dropout(0.7))
model.add(Dense(20,activation='relu'))

model.add(Dense(5,activation='relu'))


model.add(Dense(1,activation='relu'))

model.compile(loss='binary_crossentropy',
               optimizer='adam', #RMSprop(learning_rate=0.001) #SGD(lr=0.01, momentum=0.9)
               metrics=['accuracy'])

In [88]:
kfold=KFold(n_splits=5, shuffle=True,random_state=10)
scores=[]

In [89]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [90]:
for train_index, test_index in kfold.split(x):

    x_train,x_test=x.iloc[train_index],x.iloc[test_index]
    y_train,y_test=y.iloc[train_index],y.iloc[test_index]

    for i in range(5):
        model.fit(x_train,y_train,batch_size=8,epochs=1000,verbose=0) #,callbacks=[tensorboard_callback] #<- jak sie doda do modelu to pokazuje statystyki ale wolno się trenuje 
        score=model.evaluate(x_test,y_test,verbose=1)
    scores.append(score[1])
    print(score)
print('Accuracy: %.2f%% (+/- %.2f%%)' % (np.mean(scores)*100, np.std(scores)*100))

1/1 [==============================] - 0s 16ms/step - loss: 0.6114 - accuracy: 0.7500
[0.6114076972007751, 0.75]
1/1 [==============================] - 0s 16ms/step - loss: 0.6695 - accuracy: 0.7500
[0.6695348620414734, 0.75]
1/1 [==============================] - 0s 17ms/step - loss: 0.6092 - accuracy: 0.7826
[0.6092432141304016, 0.782608687877655]
1/1 [==============================] - 0s 18ms/step - loss: 0.7262 - accuracy: 0.6087
[0.7262042760848999, 0.6086956262588501]
1/1 [==============================] - 0s 17ms/step - loss: 0.6642 - accuracy: 0.7391
[0.6642099618911743, 0.739130437374115]
Accuracy: 72.61% (+/- 6.05%)


In [91]:
model.summary()
# %tensorboard --logdir logs/fit

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_4 (Dropout)         (None, 15)                0         
                                                                 
 dense_30 (Dense)            (None, 20)                320       
                                                                 
 dense_31 (Dense)            (None, 5)                 105       
                                                                 
 dense_32 (Dense)            (None, 1)                 6         
                                                                 
Total params: 431
Trainable params: 431
Non-trainable params: 0
_________________________________________________________________


In [92]:
model.save('modelV1.h5')